# Semantic Segmentation using U-NET model

## Import libraries

In [5]:
# visualization library
import cv2
from matplotlib import pyplot as plt
# data storing library
import numpy as np
import pandas as pd
# torch libraries
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset, sampler
# architecture and data split library
from sklearn.model_selection import train_test_split
# augmentation library
from albumentations import (HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
from albumentations.pytorch import ToTensorV2
# others
import os
import pdb
import time
import warnings
import random
from tqdm import tqdm_notebook as tqdm
import concurrent.futures
# warning print supression
warnings.filterwarnings("ignore")
import PIL

## Preprocessing

### Fixing seed value

In [4]:
def setSeed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

setSeed(42)

In [9]:
os.listdir('.pytorch/carvanasemseg/train_masks')[:3]

['00087a6bd4dc_01_mask.gif',
 '00087a6bd4dc_02_mask.gif',
 '00087a6bd4dc_03_mask.gif']

In [17]:
# !mkdir -p '.pytorch/carvanasemseg/'
# !mkdir -p '.pytorch/carvanasemseg/train_masks_png/'
# !mkdir -p '.pytorch/carvanasemseg/train-128/'
# !mkdir -p '.pytorch/carvanasemseg/train_masks-128/'

def convert_img(fn):
    PIL.Image.open(f'.pytorch/carvanasemseg/train_masks/{fn}').save(f'.pytorch/carvanasemseg/train_masks_png/{fn[:-4]}.png') #opening and saving image
fns = os.listdir('.pytorch/carvanasemseg/train_masks')
with concurrent.futures.ThreadPoolExecutor(8) as e:
    e.map(convert_img, fns)


def resize_mask(fn):
    PIL.Image.open('.pytorch/carvanasemseg/train_masks_png/'+fn).resize((128,128)).save(f'.pytorch/carvanasemseg/train_masks-128/{fn}')

fns = os.listdir('.pytorch/carvanasemseg/train_masks_png/')
with concurrent.futures.ThreadPoolExecutor(8) as e:
    e.map(resize_mask, fns)

# we convert the high resolution input image to 128*128
def resize_img(fn):
    PIL.Image.open('.pytorch/carvanasemseg/train/'+fn).resize((128,128)).save(f'.pytorch/carvanasemseg/train-128/{fn}')

fns = os.listdir('.pytorch/carvanasemseg/train/')
with concurrent.futures.ThreadPoolExecutor(8) as e:
    e.map(resize_img, fns)

In [18]:
df=pd.read_csv('.pytorch/carvanasemseg/train_masks.csv')
# location of original and mask image
img_fol='.pytorch/carvanasemseg/train-128/'
mask_fol='.pytorch/carvanasemseg/train_masks-128/'
# imagenet mean/std will be used as the resnet backbone is trained on imagenet stats
mean, std=(0.485, 0.456, 0.406),(0.229, 0.224, 0.225)